<a href="https://colab.research.google.com/github/JosephBless/Deep-Learning-Colab/blob/main/%E3%80%8C3_blast_msa_ipynb%E3%80%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BLAST

In this exercise, you will get to try some BLAST and multiple sequence alignment from the context of a Python program. The browser-based web tools are great resources, and we will provide links to these in parallel with the programming exercises. However, the goal here is to experience some of the flexibility that comes with being able to put together our own tools to do  specific tasks and answer particular questions that might not perfectly overlap with the tools provided on websites.

We will use a database called Swiss-Prot, that was created in 1986 by Amos Bairoch during his PhD and developed by the Swiss Institute of Bioinformatics. Its aim was to provide reliable protein sequences and a minimal level of redundancy. It is a good source for retrieving high quality records. Today it is integrated with Uni-Prot.

First run the two cells below to install biopython and download the entire Swiss-Prot database onto Google's server. Other databases can be downloaded in this same way through NCBI's file transfer page: https://ftp.ncbi.nlm.nih.gov/blast/db/ Including the entire genbank nucleotide database for example as well as specialized databases for certain genomes, viruses, etc..


In [ ]:
pip install biopython

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/swissprot.gz

Run the line below to unzip the compressed database file.

In [ ]:
!gunzip swissprot.gz

Next run the cell below to download two home-made modules that we will use in this exercise; they are based mostly on code from Migeul Rocha https://github.com/miguelfrocha/BioinformaticsAlgorithmsBook but we have added a few things for the purposes of visualization and the goals of this exercise. The first module will help us perform multiple sequence alignments and visualize them, and the second is a simplified BLAST program (which does not utilize Blosum or a substitution matrix - only a fixed match/mismatch score) that we will be able to run locally.

In [ ]:
import sys
from os import path

if path.exists("/content/bioinformatics_stockholm/"):
  pass
else:
  !git clone https://github.com/Intertangler/bioinformatics_stockholm
sys.path.insert(0,'/content')

import importlib.util
spec = importlib.util.spec_from_file_location("msa", "/content/bioinformatics_stockholm/msa/msa.py")
msa = importlib.util.module_from_spec(spec)
spec.loader.exec_module(msa)

spec = importlib.util.spec_from_file_location("miniblast", "/content/bioinformatics_stockholm/Miniblast/miniblast.py")
miniblast = importlib.util.module_from_spec(spec)
spec.loader.exec_module(miniblast)


## web BLAST
First, if you have never used it before, check out the web-based BLAST hosted by the NIH. https://blast.ncbi.nlm.nih.gov/Blast.cgi Note the multiple types of BLAST that you can choose - for example nucleotide or protein-based depending on the type of sequence you will use for your query. If you go to protein blast, you will see an input box that accepts sequences or accession numbers. Note the various options such as choice of database (the standard one for protein searches is called "nr"). You can try searching for this sequence for example: "MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKALVLIAFAQYLQQCPFEDHVKLVNEVTEFAKTCVADESAENCDKSLHTLFGDKLCTVATLRETYGEMADCCAKQEPERNECFLQHKDDNPNLPRLVRPEVDVMCTAFHDNEETFLKKYLYEIA". You should get several hits related to the protein albumin. Their significance level should be high, indicated by correspondingly low E-values (expected number of random matches to your input).

## local BLAST
Now that we've tried the internet-based version, let's do some local BLASTing on the Swiss-Prot database that we have downloaded. First run the cell below to define our Blosum62 matrix that will be used for our alignments.

In [ ]:
blosum62 = {
    ('W', 'F'): 1, ('L', 'R'): -2, ('S', 'P'): -1, ('V', 'T'): 0,
    ('Q', 'Q'): 5, ('N', 'A'): -2, ('Z', 'Y'): -2, ('W', 'R'): -3,
    ('Q', 'A'): -1, ('S', 'D'): 0, ('H', 'H'): 8, ('S', 'H'): -1,
    ('H', 'D'): -1, ('L', 'N'): -3, ('W', 'A'): -3, ('Y', 'M'): -1,
    ('G', 'R'): -2, ('Y', 'I'): -1, ('Y', 'E'): -2, ('B', 'Y'): -3,
    ('Y', 'A'): -2, ('V', 'D'): -3, ('B', 'S'): 0, ('Y', 'Y'): 7,
    ('G', 'N'): 0, ('E', 'C'): -4, ('Y', 'Q'): -1, ('Z', 'Z'): 4,
    ('V', 'A'): 0, ('C', 'C'): 9, ('M', 'R'): -1, ('V', 'E'): -2,
    ('T', 'N'): 0, ('P', 'P'): 7, ('V', 'I'): 3, ('V', 'S'): -2,
    ('Z', 'P'): -1, ('V', 'M'): 1, ('T', 'F'): -2, ('V', 'Q'): -2,
    ('K', 'K'): 5, ('P', 'D'): -1, ('I', 'H'): -3, ('I', 'D'): -3,
    ('T', 'R'): -1, ('P', 'L'): -3, ('K', 'G'): -2, ('M', 'N'): -2,
    ('P', 'H'): -2, ('F', 'Q'): -3, ('Z', 'G'): -2, ('X', 'L'): -1,
    ('T', 'M'): -1, ('Z', 'C'): -3, ('X', 'H'): -1, ('D', 'R'): -2,
    ('B', 'W'): -4, ('X', 'D'): -1, ('Z', 'K'): 1, ('F', 'A'): -2,
    ('Z', 'W'): -3, ('F', 'E'): -3, ('D', 'N'): 1, ('B', 'K'): 0,
    ('X', 'X'): -1, ('F', 'I'): 0, ('B', 'G'): -1, ('X', 'T'): 0,
    ('F', 'M'): 0, ('B', 'C'): -3, ('Z', 'I'): -3, ('Z', 'V'): -2,
    ('S', 'S'): 4, ('L', 'Q'): -2, ('W', 'E'): -3, ('Q', 'R'): 1,
    ('N', 'N'): 6, ('W', 'M'): -1, ('Q', 'C'): -3, ('W', 'I'): -3,
    ('S', 'C'): -1, ('L', 'A'): -1, ('S', 'G'): 0, ('L', 'E'): -3,
    ('W', 'Q'): -2, ('H', 'G'): -2, ('S', 'K'): 0, ('Q', 'N'): 0,
    ('N', 'R'): 0, ('H', 'C'): -3, ('Y', 'N'): -2, ('G', 'Q'): -2,
    ('Y', 'F'): 3, ('C', 'A'): 0, ('V', 'L'): 1, ('G', 'E'): -2,
    ('G', 'A'): 0, ('K', 'R'): 2, ('E', 'D'): 2, ('Y', 'R'): -2,
    ('M', 'Q'): 0, ('T', 'I'): -1, ('C', 'D'): -3, ('V', 'F'): -1,
    ('T', 'A'): 0, ('T', 'P'): -1, ('B', 'P'): -2, ('T', 'E'): -1,
    ('V', 'N'): -3, ('P', 'G'): -2, ('M', 'A'): -1, ('K', 'H'): -1,
    ('V', 'R'): -3, ('P', 'C'): -3, ('M', 'E'): -2, ('K', 'L'): -2,
    ('V', 'V'): 4, ('M', 'I'): 1, ('T', 'Q'): -1, ('I', 'G'): -4,
    ('P', 'K'): -1, ('M', 'M'): 5, ('K', 'D'): -1, ('I', 'C'): -1,
    ('Z', 'D'): 1, ('F', 'R'): -3, ('X', 'K'): -1, ('Q', 'D'): 0,
    ('X', 'G'): -1, ('Z', 'L'): -3, ('X', 'C'): -2, ('Z', 'H'): 0,
    ('B', 'L'): -4, ('B', 'H'): 0, ('F', 'F'): 6, ('X', 'W'): -2,
    ('B', 'D'): 4, ('D', 'A'): -2, ('S', 'L'): -2, ('X', 'S'): 0,
    ('F', 'N'): -3, ('S', 'R'): -1, ('W', 'D'): -4, ('V', 'Y'): -1,
    ('W', 'L'): -2, ('H', 'R'): 0, ('W', 'H'): -2, ('H', 'N'): 1,
    ('W', 'T'): -2, ('T', 'T'): 5, ('S', 'F'): -2, ('W', 'P'): -4,
    ('L', 'D'): -4, ('B', 'I'): -3, ('L', 'H'): -3, ('S', 'N'): 1,
    ('B', 'T'): -1, ('L', 'L'): 4, ('Y', 'K'): -2, ('E', 'Q'): 2,
    ('Y', 'G'): -3, ('Z', 'S'): 0, ('Y', 'C'): -2, ('G', 'D'): -1,
    ('B', 'V'): -3, ('E', 'A'): -1, ('Y', 'W'): 2, ('E', 'E'): 5,
    ('Y', 'S'): -2, ('C', 'N'): -3, ('V', 'C'): -1, ('T', 'H'): -2,
    ('P', 'R'): -2, ('V', 'G'): -3, ('T', 'L'): -1, ('V', 'K'): -2,
    ('K', 'Q'): 1, ('R', 'A'): -1, ('I', 'R'): -3, ('T', 'D'): -1,
    ('P', 'F'): -4, ('I', 'N'): -3, ('K', 'I'): -3, ('M', 'D'): -3,
    ('V', 'W'): -3, ('W', 'W'): 11, ('M', 'H'): -2, ('P', 'N'): -2,
    ('K', 'A'): -1, ('M', 'L'): 2, ('K', 'E'): 1, ('Z', 'E'): 4,
    ('X', 'N'): -1, ('Z', 'A'): -1, ('Z', 'M'): -1, ('X', 'F'): -1,
    ('K', 'C'): -3, ('B', 'Q'): 0, ('X', 'B'): -1, ('B', 'M'): -3,
    ('F', 'C'): -2, ('Z', 'Q'): 3, ('X', 'Z'): -1, ('F', 'G'): -3,
    ('B', 'E'): 1, ('X', 'V'): -1, ('F', 'K'): -3, ('B', 'A'): -2,
    ('X', 'R'): -1, ('D', 'D'): 6, ('W', 'G'): -2, ('Z', 'F'): -3,
    ('S', 'Q'): 0, ('W', 'C'): -2, ('W', 'K'): -3, ('H', 'Q'): 0,
    ('L', 'C'): -1, ('W', 'N'): -4, ('S', 'A'): 1, ('L', 'G'): -4,
    ('W', 'S'): -3, ('S', 'E'): 0, ('H', 'E'): 0, ('S', 'I'): -2,
    ('H', 'A'): -2, ('S', 'M'): -1, ('Y', 'L'): -1, ('Y', 'H'): 2,
    ('Y', 'D'): -3, ('E', 'R'): 0, ('X', 'P'): -2, ('G', 'G'): 6,
    ('G', 'C'): -3, ('E', 'N'): 0, ('Y', 'T'): -2, ('Y', 'P'): -3,
    ('T', 'K'): -1, ('A', 'A'): 4, ('P', 'Q'): -1, ('T', 'C'): -1,
    ('V', 'H'): -3, ('T', 'G'): -2, ('I', 'Q'): -3, ('Z', 'T'): -1,
    ('C', 'R'): -3, ('V', 'P'): -2, ('P', 'E'): -1, ('M', 'C'): -1,
    ('K', 'N'): 0, ('I', 'I'): 4, ('P', 'A'): -1, ('M', 'G'): -3,
    ('T', 'S'): 1, ('I', 'E'): -3, ('P', 'M'): -2, ('M', 'K'): -1,
    ('I', 'A'): -1, ('P', 'I'): -3, ('R', 'R'): 5, ('X', 'M'): -1,
    ('L', 'I'): 2, ('X', 'I'): -1, ('Z', 'B'): 1, ('X', 'E'): -1,
    ('Z', 'N'): 0, ('X', 'A'): 0, ('B', 'R'): -1, ('B', 'N'): 3,
    ('F', 'D'): -3, ('X', 'Y'): -1, ('Z', 'R'): 0, ('F', 'H'): -1,
    ('B', 'F'): -3, ('F', 'L'): 0, ('X', 'Q'): -1, ('B', 'B'): 4
}
Blosum62 = {}
for i in blosum62:
  Blosum62[i] = blosum62[i]
  Blosum62[i[::-1]] = blosum62[i] #adds the reverse pairs so that we can lookup ('F', 'H') or ('H', 'F') for example

Next we will try BLASTing for the same albumin sequence we used in the web-BLAST. "MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKALVLIAFAQYLQQCPFEDHVKLVNEVTEFAKTCVADESAENCDKSLHTLFGDKLCTVATLRETYGEMADCCAKQEPERNECFLQHKDDNPNLPRLVRPEVDVMCTAFHDNEETFLKKYLYEIA".
The cell below will first run a function miniblast.read_database_fasta() which will access the lines of the Swiss-Prot database residing on the server. Then the miniblast.best_alignment_fasta() function will execute the search, with a query sequence, the database, and the word- or k-mer size as input parameters. The resulting output will be attached to a variable we call result, and in a subsequent loop we can print out the contents of the result.

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import pairwise2
import numpy as np
db = miniblast.read_database_fasta('/content/swissprot')
query = "MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKALVLIAFAQYLQQCPFEDHVKLVNEVTEFAKTCVADESAENCDKSLHTLFGDKLCTVATLRETYGEMADCCAKQEPERNECFLQHKDDNPNLPRLVRPEVDVMCTAFHDNEETFLKKYLYEIA"
result = miniblast.best_alignment_fasta(db,query,w=3)
print(result)


The database object db will return the result of an entry if the position of the entry i is used as an index.

In [ ]:
for i in result:
  print(i)
  print(db[i])

Note how we get some additional hits apparently unrelated to our query (which we already know to be albumin). To better understand which hits are genuine, it is always important to examine the alignments directly. This miniblast module does not perform the final Smith-Waterman local alignment typical in the BLAST workflow, so we can again make use of biopython's pairwise2.align() function in order to perform the final local alignment on the winning results with our query. We will simply set up a loop that goes through the database, looking up the indices produced by the BLAST function above, and which performs alignment with our query and formatting to visually view the alignments like we did with the NGS data last time.

If we check the relative scores and visually inspect each alignment now, we see that albumin (indicated where it says RecName: Full=Albumin) is indeed our winner, as the other candidates have only sparse agreement with a lot of gaps.

In [ ]:
for i in result:
  print(db[i])
  target_sequence = db[i].seq
  aln = pairwise2.align.localds(query,target_sequence, match_dict = Blosum62, open = -3, extend = -1)[0]
  print(pairwise2.format_alignment(align1=aln[0], align2=aln[1], score=aln[2], begin=aln[3], end=aln[4], full_sequences=True))

## Problem 3.1

Consider a hypothetical scenario: the following sequences are fragmented data that have been isolated from a pathogen whose current identity is unknown. It is believed that the sequences are all derived from a single protein related to the pathogen. To make things interesting, we've tossed in a few totally random sequences to simulate a bit of experimental noise. Thus, you could go through all the sequences and web-BLAST them individually, but a more convenient approach will be to write a program that systematically performs miniblast on our Swiss-Prot database, and which at the same time generates alignments for you to check as we did above. After running the program, visually inspect the results or locate the high-scoring alignments and determine the protein of interest. **What is the protein's name? For your submission, report the answer as the name of the entry, i.e. where it says RecName: Full=.... in the database record** Did you discover the identity of the pathogen? Note that the BLAST function can take a few minutes to run with multiple queries.

fragments = ["EHFTGQLPMKQVTATIMIRNCEMEIRHIWCMREIVFEETQGEM","VSWEDPLVISTDANFCGNHWYPSTQDHEACNKF","FWMLMIYNVAGNKWVTVYYGVPVWKEAKTTLFCASDAKGY","TTEYFGKGIPLYRKFEWSDPPSPFSAKQTYHWGKMTTSYYFDH","EREVHNIWATHACVPTDPSPQEMFLVNVTENFNMWKNDMVDQMHEDIISL","THGIKPVVSTQLLLNGSLAEEEIIIRSENLTDNAKTIIVHLNESIEIECVRPNNNTRKSVRIGPGQTFYATGDIIGDIRAAYCDINKDKWNRALQRVSKKLRELYPNTTTIH","DKWNRALQRVSKKLRELYPNTTTIHFNSSSGGDPEITMHSFNCRGEFFYCDTSRLFNGTYNGTDSNPNNGTDSDSNSNSNITLPCRIKQIINMWQEVGRAMYAPPIAGSITCKSNITGLLLTRDGGKEENSTTNRIEIFRPGGGNMKDN","LSQNQQEQNEKDLLSLDKWQTLWNWFDITNWLWYIKIFIMIVGGLIGLRIIFAVLSIINRVRQGYSPLSLQTLIPNPREPDRLGRIEEEGGEQDRNRSIRLVNGFLALAWDDLRNLCLFSYHQLRNFILIVARAVELLGRSSLRGLQRGWEAL", "FLALAWDDLRNLCLFSYHQLRNFILIVARAVELLGRSSLRGLQRGWEALKYLGSIVQYWGLELKKSAVSLFDAIAIAVAEGTDRII", "FKGRWKESSMPYFWMFQHADTSIIIWFCKMWCPTRTTSFKT"]

# multiple sequence alignment

The code below will performa multiple sequence alignment on a dictionary of input sequences (with their names or ID's as keys, and their sequences as values). Try running it for the test sequences below and have a look at the output alignment object that is produced, as well as the color interactive visualization we have made below that. The next problem will simply be a matter of replacing the input sequences and rerunning this program.

In case you are curious, these sequences are from the zinc finger family that contains members involved in the regulation of transcription processes. Zinc finger domains play a role in the mediation of protein-DNA interactions. The multiple sequence alignment reveals to us some of the common/conserved structure necessary for this function.

In [ ]:
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment

example_seqs = {
"ATRX_HUMAN/159-296" :     "KRGEDGLHGIVSCTACGQQVNHFQKDSIYRHPSLQVLICKNCFKYYMSDDISRDSDGMDEQCRWCAEGGNLICCDFCHNAFCKKCILRNLGRKELSTIMDENNQWYCYICHPEPLLDLVTACNSVFENLEQLLQQNKK",
"ATRX_MACEU/21-158"  :     "KKRGEGLHGIVSCTACGQQVNHFQKDSIYRHPTLKVLICKNCYKYYMSDDISRDADGMDEQCRWCAEGGNLICCDFCHNAFCKKCILRNLGRKELSAIMDENSQWYCYICRPEPLLDLVTACHSVFKNLEQLLQQNKK",
"ATRX_MOUSE/158-295" :     "KRGEDGLHGIVSCTACGQQVNHFQKDSIYRHPSLKVLICKNCFKYYMSDDISRDSDGMDEQCRWCAEGGNLICCDFCHNAFCKKCILRNLGRKELSTIMDENNQWYCYICQPEPLLDLVTACNSVFENLEQLLQQNKK",
"ATRX_PANTR/159-296" :     "KRGEDGLHGIVSCTACGQQVNHFQKDSIYRHPSLQVLICKNCFKYYMSDDISRDSDGMDEQCRWCAEGGNLICCDFCHNAFCKKCILRNLGRKELSTIMDENNQWYCYICHPEPLLDLVTACNSVFENLEQLLQQNKK",
"ATRX_PONPY/159-296" :     "KRGEDGLHGIVSCTACGQQVNHFQKDSIYRHPSLQVLICKNCFKYYMSDDISRDSDGMDEQCRWCAEGGNLICCDFCHNAFCKKCILRNLGRKELSTIMDENNQWYCYICHPEPLLDLVTACNSVFDNLEQLLQQNKK",
"ATRY_MACEU/1-96"    :     "VICTACGQQVNQFQKDSIYRHPTLNVLICKRWCAEGGNLICCDSCHNAFCKKCIWRNLGRKEISKIMNEKNEWHCYICCPEPLLDLIAVCDSVLEN",
"CHR20_ARATH/472-601":     "RDDSQNPANNFRCTACNKVAVEVHSHPLLEVIVCMDCKRSIEDRVSKVDDSLERHCEWCGHIADLIDCRTCEKLFCASCIKRNIGEEYMSEAQSSGWDCCCCSPIPLQRLTLELEKAMRDKKSIELSSDS",
"DNM3A_CHICK/447-579":     "EVRQKCRNIEDICISCGSLNVTLEHPLFIGGMCQNCKNCFLECAYQYDDDGYQSYCTICCGGREVLMCGNNNCCRCFCVECVDLLVGPGAAQAAIKEDPWNCYMCGHKGVYGLLRRREDWPSRLQMFFANNHD",
"DNM3A_HUMAN/482-614":     "EVRQKCRNIEDICISCGSLNVTLEHPLFVGGMCQNCKNCFLECAYQYDDDGYQSYCTICCGGREVLMCGNNNCCRCFCVECVDLLVGPGAAQAAIKEDPWNCYMCGHKGTYGLLRRREDWPSRLQMFFANNHD",
"DNM3A_MOUSE/478-610":     "EVRQKCRNIEDICISCGSLNVTLEHPLFIGGMCQNCKNCFLECAYQYDDDGYQSYCTICCGGREVLMCGNNNCCRCFCVECVDLLVGPGAAQAAIKEDPWNCYMCGHKGTYGLLRRREDWPSRLQMFFANNHD",
"DNM3A_RAT/478-610"  :     "EVRQKCRNIEDICISCGSLNVTLEHPLFIGGMCQNCKNCFLECAYQYDDDGYQSYCTVCCEGRELLLCSNTSCCRCFCVECLEVLVGTGTAAEAKLQEPWSCYMCLPQRCHGVLRRREDWPSRLQMFFANNHD",
"DNM3B_HUMAN/423-555":     "DVANNKSSLEDGCLSCGRKNPVSFHPLFEGGLCQTCRDRFLELFYMYDDDGYQSYCTVCCEGRELLLCSNTSCCRCFCVECLEVLVGAGTAEDAKLQEPWSCYMCLPQRCHGVLRRRKDWNVRLQAFFTSDTG",
"DNM3B_MOUSE/428-560":     "EVTNNKGNLEDRCLSCGKKNPVSFHPLFEGGLCQSCRDRFLELFYMYDEDGYQSYCTVCCEGRELLLCSNTSCCRCFCVECLEVLVGAGTAEDVKLQEPWSCYMCLPQRCHGVLRRRKDWNMRLQDFFTTDPD",
"DNM3C_MOUSE/309-441":     "DVTNNKGNLEDHCLSCGRKDPVSFHPLFEGGLCQSCRDRFLELFYMYDEDGYQSYCSICCSGETLLICGNPDCTRCYCFECVDSLVGPGTSGKVHAMSNWVCYLCLPSSRSGLLQRRKDWNMRLQDFFTTDPD",
"DNM3L_HUMAN/41-173" :     "EVKANQRNIEDICICCGSLQVHTQHPLFEGGICAPCKDKFLDALFLYDDDGYQSYCTICCSGGTLFICESPDCTRCYCFECVDILVGPGTSERINAMACWVCFLCLPFSRSGLLQRRRKWRSQLKAFYDRESE",
"DNM3L_MOUSE/75-207" :     "EVKVNRRSIEDICLCCGTLQVYTRHPLFEGGLCAPCKDKFLESLFLYDDDGHQSYCTICCSGHTLFICESPDCTRCYCFECVDILVGPGTSERINAMACWVCFLCLPFSRSGLLQRRKRWRHQLKAFHDQEGA",
"DNM3L_RAT/76-208"   :     "EVNVNQRNIEDICLCCGSLQVYAQHPLFEGGICAPCKDKFLETLFLYDEDGHQSYCTICCGGREVLMCGNNNCCRCFCVECVDLLVGPGAAQAAIKEDPWNCYMCGHKGTYGLLRRRKKWRHQLKAFHDREGA",
}



Example_seqs = [msa.MySeq(val, "PROTEIN") for val in example_seqs.values()]

sm = msa.SubstMatrix()
# sm.create_submat(1, -1, "ACGT")
sm.read_submat_file("/content/bioinformatics_stockholm/msa/blosum62.mat", "\t")
aseq = msa.PairwiseAlignment(sm, -1)
ma = msa.MultipleAlignment(Example_seqs, aseq)


The following code will take the alignment object produced in the last cell and save it to the server as a FASTA file. Note the filename "multiple_alignment_example.fsa" as the destination for the data. Each fasta entry is a sequence "record" that gets made here by supplying a sequence and an id.

The FASTA format is simply a greater-than sign followed by identifying information, and then sequence information below that (for example):

$> SEQUENCE 1$
$MTEITAAMVKELRESTGAGMMDCKNALSETNGDFDKAVQLLREKGLGKAAKKADRLAAEG$
$LVSVKVSDDFTIAAMRPSYLSYEDLDMTFVENEYKALVAELEKENEERRRLKDPNKPEHK$
$IPQFASRKQLSDAILKEAEEKIKEELKAQGKPEKIWDNIIPGKMNSFIADNSQLDSKLTL$
$MGQFYVMDDKKTVEQVIAEKEKEFGGKIKIVEFICFEVGEGLEKKTEDFAAEVAAQL$

And the file output will basically be a text file with entries that look like this. It is the same format we used for our NGS data in the previous exercises.

In [ ]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import numpy as np
i = 0
with open("/content/bioinformatics_stockholm/multiple_alignment_example.fsa", 'w') as filetowrite:
    filetowrite.write('')
for seq in ma.align_consensus():
  # print(seq)
  sorted_rec = SeqRecord(Seq(seq),
                          id=str(list(example_seqs.keys())[i]),
                          name=str(list(example_seqs.keys())[i]),
                          description=f"sequence: {i}")
  print(sorted_rec.seq)
  with open("/content/bioinformatics_stockholm/multiple_alignment_example.fsa", "a", ) as destination:
    SeqIO.write(sorted_rec, destination, "fasta")
  i += 1


The code below will take the FASTA file produced in the previous step and produce an interactive plot that can be used to visualize the multiple alignment. You may click and drag the window to scroll through the positions.

In [ ]:
import panel as pn
pn.extension()
aln = AlignIO.read("/content/bioinformatics_stockholm/multiple_alignment_example.fsa", "fasta")
p = msa.view_alignment(aln, plot_width=900)
pn.pane.Bokeh(p)


## Problem 3.2
Cas9 proteins are part of the CRISPR bacterial defense system against bacteria-infecting viruses, and Cas9 proteins are common throughout the bacterial kingdom. The proteins vary widely in size and sequence, however all known Cas9 enzymes contain a domain that cleaves the DNA strand complementary to the guide RNA. The following sequences were published in 2014 by Jennifer Doudna and colleagues, and their multiple sequence analysis of this collection led to their discovery of the protein's function, a discovery which was awarded the Nobel prize in 2020. **Use the scripts demonstrated above to perform a multiple sequence alignment on these sequences. Which amino acid (single letter code) is most represented in column/position 99 in the resulting multiple sequence alignment you get?** Note, you should not change the order of the sequences from how they are represented now. We will explore why this is the case later - but essentially the result may change if you alter the order.

"CAS9_ACTNH/513-675": "GTTIGYHTCQLDHIVPQAGPGSNNRRGNLVAVCERCNRSKSNTPFAVWAQKCGIPGTTIGYHTCQLDHIVPQAGPGSNNRRGNLVAVCERCNRSKSNTPFAVWAQKCGIPHVGVKEAIGRVRGWRKQTPNTSSEDLTRLKKEVIARLRRTQEDPEIDERSM",
"CAS9A_STRTD/516-684": "CLYTGKTISIHDLINNSNQFEVDHILPLSITFDDSLANKVLVYATANQEKGQRTPYQALDCLYTGKTISIHDLINNSNQFEVDHILPLSITFDDSLANKVLVYATANQEKGQRTPYQALDSMDDAWSFRELKAFVRESKTLSNKKKEYLLTEEDISKFDVRKKFIERNL",
"CAS9B_STRTD/771-928": "LQNGKDMYTGDDLDIDRLSNYDIDHIIPQAFLKDNSIDNKVLVSSASNRGKSDDVPSLLQNGKDMYTGDDLDIDRLSNYDIDHIIPQAFLKDNSIDNKVLVSSASNRGKSDDVPSLEVVKKRKTFWYQLLKSKLISQRKFDNLTKAERGGLSPEDKAGFIQRQL",
"CAS9_CAMJE/487-637": "KIKISDLQDEKMLEIDHIYPYSRSFDDSYMNKVLVFTKQNQEKLNQTPFEAFGNDSAKWKIKISDLQDEKMLEIDHIYPYSRSFDDSYMNKVLVFTKQNQEKLNQTPFEAFGNDSAKWQKIEVLAKNLPTKKQKRILDKNYKDKEQKNFKDRNL",
"CAS9_CORDI/504-665": "GSPITFSNSEMDHIVPRAGQGSTNTRENLVAVCHRCNQSKGNTPFAIWAKNTSIEGSPITFSNSEMDHIVPRAGQGSTNTRENLVAVCHRCNQSKGNTPFAIWAKNTSIEGVSVKEAVERTRHWVTDTGMRSTDFKKFTKAVVERFQRATMDEEIDARSM",
"CAS9_FRATN/897-1046": "NLTDGDFDGAKEELDHIIPRSHKKYGTLNDEANLICVTRGDNKNKGNRIFCLRDLADNYNLTDGDFDGAKEELDHIIPRSHKKYGTLNDEANLICVTRGDNKNKGNRIFCLRDLADNYKLKQFETTDDLEIEKKIADTIWDANKKDFKFGNY",
"CAS9_LISIN/773-924": "MYTGQDLDIHNLSNYDIDHIVPQSFITDNSIDNLVLTSSAGNREKGDDVPPLEIVRKRMYTGQDLDIHNLSNYDIDHIVPQSFITDNSIDNLVLTSSAGNREKGDDVPPLEIVRKRKVFWEKLYQGNLMSKRKFDYLTKAERGGLTEADKARFIHRQL",
"CAS9_NEIM8/512-667": "SGKEINLGRLNEKGYVEIDHALPFSRTWDDSFNNKVLVLGSENQNKGNQTPYEYFNGKDNSGKEINLGRLNEKGYVEIDHALPFSRTWDDSFNNKVLVLGSENQNKGNQTPYEYFNGKDNSREWQEFKARVETSRFPRSKKQRILLQKFDEDGFKERNL",
"CAS9_NEIMA/512-667": "SGKEINLGRLNEKGYVEIDHALPFSRTWDDSFNNKVLVLGSENQNKGNQTPYEYFNGKDNSGKEINLGRLNEKGYVEIDHALPFSRTWDDSFNNKVLVLGSENQNKGNQTPYEYFNGKDNSREWQEFKARVETSRFPRSKKQRILLQKFDEDGFKERNL",
"CAS9_PASMU/504-659": "SGKEINIHRLNEKGYVEIDHALPFSRTWDDSFNNKVLVLASENQNKGNQTPYEWLQGKINSGKEINIHRLNEKGYVEIDHALPFSRTWDDSFNNKVLVLASENQNKGNQTPYEWLQGKINSERWKNFVALVLGSQCSAAKKQRLLTQVIDDNKFIDRNL",
"CAS9_STAAU/480-646": "LEAIPLEDLLNNPFNYEVDHIIPRSVSFDNSFNNKVLVKQEENSKKGNRTPFQYLSSSDSLEAIPLEDLLNNPFNYEVDHIIPRSVSFDNSFNNKVLVKQEENSKKGNRTPFQYLSSSDSKISYETFKKHILNLAKGKGRISKTKKEYLLEERDINRFSVQKDFINRNL",
"CAS9_STRMU/770-921": "MYTGEELDIDYLSQYDIDHIIPQAFIKDNSIDNRVLTSSKENRGKSDDVPSKDVVRKMMYTGEELDIDYLSQYDIDHIIPQAFIKDNSIDNRVLTSSKENRGKSDDVPSKDVVRKMKSYWSKLLSAKLITQRKFDNLTKAERGGLTDDDKAGFIKRQL",
"CAS9_STRP1/770-921": "MYVDQELDINRLSDYDVDHIVPQSFLKDDSIDNKVLTRSDKNRGKSDNVPSEEVVKKMMYVDQELDINRLSDYDVDHIVPQSFLKDDSIDNKVLTRSDKNRGKSDNVPSEEVVKKMKNYWRQLLNAKLITQRKFDNLTKAERGGLSELDKAGFIKRQL",
"CAS9_STRTR/792-949": "LQNGKDMYTGDDLDIDRLSNYDIDHIIPQAFLKDNSIDNKVLVSSASNRGKSDDFPSLLQNGKDMYTGDDLDIDRLSNYDIDHIIPQAFLKDNSIDNKVLVSSASNRGKSDDFPSLEVVKKRKTFWYQLLKSKLISQRKFDNLTKAERGGLLPEDKAGFIQRQL"

## Problem 3.3
The following set of sequences are from the Nsp3e nucleic acid-binding domain of betacoronaviruses - the family includes members like SARS, SARS-CoV-2, BatCoV RaTG13 and Bat-SARS-like coronavirus (BATSL-CoVZXC21 and BAT-SL-CoVZC45).

**Generate a "dumb" consensus for the Nsp3e sequences.**  First run the code below, and it will create a list the_sequences = ma.align_consensus() which is a list of each of the aligned (gaps inserted) sequences in the multiple sequence alignment. Your task is to figure out how to determine which base is the most prevalent in each column, and print out a final consensus sequence. **For canvas: what is the consensus sequence? Include any gaps in this answer.** To help you out, we have provided a small function called most_frequent() which will check which is the most-prevalent character of a *list* or *array* of characters. For example if you give it ["A", "A", "C"] it will return the string "A".

(note again - do not change the order of sequences from how they are currently listed as this will affect the result of the alignment - we will discuss why later).


Additional guidance: Wondering where to start? This problem is a matter of knowing how to index/slice arrays, lists, and strings in an intermediate/advanced way. The goal is essentially to fetch the columns of this multiple sequence alignment, however the problem is that the sequences are provided as a list of linear strings.

In [ ]:
def most_frequent(List):
    List = list(List)
    return max(set(List), key = List.count)

In [ ]:
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment

Nsp3e_seqs = {
"R1AB_BC133/1885-2002":"TIKYSPATILPGSVYSNSCLVGVDGTPGSDTISKFFNDLLGFDETKPISKKLTYSLLPNEDGDVLLSEFNNYNPVYKKGVMLKGKPILWVNNGVCDSALNKPNRASLRQLYDVAPIVL",
"R1AB_BC279/1894-2004":"PIDLVPTQPLPNASFDNFKLTCSNTKFADDLNQMTGFKKPASRELSVTFFPDLNGDVVAIDYRHYSASFKKGAKLLHKPIIWHINQTTNKTTYKPNTWCLRCLWSTKPVET",
"R1AB_BCHK3/1882-1992":"PIDLVPTQPMPNASFDNFKLTCSNTKFADDLNQMTGFKKPASRELTVTFFPDLNGDVVAIDYRHYSTSFKKGAKLVHKPILWHINQTTNKTTYKPNIWCLRCLWSTKPVDT",
"R1AB_BCHK4/1878-1995":"TIKYSPATILPGSVYSNSCLVGVDGTPGSDTISKFFNDLLGFDETKPISKKLTYSLLPNEDGDVLLSEFSNYNPVYKKGVMLKGKPILWVNNGVCDSALNKPNRASLRQLYDVAPIVL",
"R1AB_BCHK5/1916-2033":"VIEYSPATILSGSVYTNSCLVGHDGTIGSDAISSSFNNLLGFDNSKPVSKKLTYSFFPDFEGDVILTEYSTYDPIYKNGAMLHGKPILWVNNSKFDSALNKFNRATLRQVYDIAPVTL",
"R1AB_BCHK9/1770-1870":"PIEVVAAPKLVTSYDGFYLSSCQNPQLAESFNKAINATKTGPMKLLTMYPNVAGDVVAISDDNVVAHPYGSLHMGKPVLFVTRPNTWKKLVPLLSTVVVNT",
"R1AB_BCRP3/1886-1996":"PIDLIPTQPLPNASFDNFKLTCSNTKFADDLNQMTGFTKPASRELSVTFFPDLNGDVVAIDYRHYSASFKKGAKLLHKPIVWHINQATTKTTFKPNTWCLRCLWSTKPVDT",
"R1A_BC133/1885-2002":"TIKYSPATILPGSVYSNSCLVGVDGTPGSDTISKFFNDLLGFDETKPISKKLTYSLLPNEDGDVLLSEFNNYNPVYKKGVMLKGKPILWVNNGVCDSALNKPNRASLRQLYDVAPIVL",
"R1A_BC279/1894-2004":"PIDLVPTQPLPNASFDNFKLTCSNTKFADDLNQMTGFKKPASRELSVTFFPDLNGDVVAIDYRHYSASFKKGAKLLHKPIIWHINQTTNKTTYKPNTWCLRCLWSTKPVET",
"R1A_BCHK3/1882-1992":"PIDLVPTQPMPNASFDNFKLTCSNTKFADDLNQMTGFKKPASRELTVTFFPDLNGDVVAIDYRHYSTSFKKGAKLVHKPILWHINQTTNKTTYKPNIWCLRCLWSTKPVDT",
"R1A_BCHK4/1878-1995":"TIKYSPATILPGSVYSNSCLVGVDGTPGSDTISKFFNDLLGFDETKPISKKLTYSLLPNEDGDVLLSEFSNYNPVYKKGVMLKGKPILWVNNGVCDSALNKPNRASLRQLYDVAPIVL",
"R1A_BCHK5/1916-2033":"VIEYSPATILSGSVYTNSCLVGHDGTIGSDAISSSFNNLLGFDNSKPVSKKLTYSFFPDFEGDVILTEYSTYDPIYKNGAMLHGKPILWVNNSKFDSALNKFNRATLRQVYDIAPVTL",
"R1A_BCHK9/1770-1870":"PIEVVAAPKLVTSYDGFYLSSCQNPQLAESFNKAINATKTGPMKLLTMYPNVAGDVVAISDDNVVAHPYGSLHMGKPVLFVTRPNTWKKLVPLLSTVVVNT",
"R1A_BCRP3/1886-1996":"PIDLIPTQPLPNASFDNFKLTCSNTKFADDLNQMTGFTKPASRELSVTFFPDLNGDVVAIDYRHYSASFKKGAKLLHKPIVWHINQATTKTTFKPNTWCLRCLWSTKPVDT",
"R1AB_CVBEN/1906-2007":"IKAQFKTFEKVDGVYTNFKLIGHTICDILNAKLGFDSSKEFVEYKVTEWPTATGDVVLATDDLYVKRYERGCITFGKPVIWLSHEQASLNSLTYFNRPLLVD",
"R1AB_CVBLU/1906-2007":"IKAQFKTFEKVDGVYTNFKLIGHTICDILNAKLGFDSSKEFVEYKVTEWPTATGDVVLATDDLYVKRYERGCITFGKPVIWLSHEQASLNSLTYFNRPLLVD",
"R1AB_CVBM/1906-2007":"IKAQFKTFEKVDGVYTNFKLIGHTVCDILNAKLGFDSSKEFVEYKVTEWPTATGDVVLATDDLYVKRYERGCITFGKPVIWLSHEQASLNSLTYFNRPLLVD",
"R1AB_CVBQ/1906-2007":"IKAQFKTFEKVDGVYTNFKLIGHTVCDILNAKLGFDSSKEFVEYKVTEWPTATGDVVLATDDLYVKRYERGCITFGKPVIWLSHEQASLNSLTYFNRPLLVD",
"R1AB_CVHN1/1992-2093":"IKAQFKPFAKVDGVYTNFKLVGHDICAQLNDKLGFNVDLPFVEYKVTVWPVATGDVVLASDDLYVKRYFKGCETFGKPVIWFCHDEASLNSLTYFNKPSFKS",
"R1AB_CVHN2/1962-2063":"IKAQFKPFAKVDGVYTNFKLVGHDICAQLNDKLGFNVDLPFVEYKVTVWPVATGDVVLASDDLYVKRYFKGCETFGKPVIWLCHDEASLNSLTYFNKPSFKS",
"R1AB_CVHN5/1942-2043":"IKAQFKPFAKVDGVYTNFKLVGHDICAQLNDKLGFNVDLPFVEYKVTVWPVATGDVVLASDDLYVKRYFKGCETFGKPVIWFCHDEASLNSLTYFNKPSFKS",
"R1AB_CVHOC/1906-2007":"IKAQFKTFEKVDGVYTNFKLIGHTVCDSLNSKLGFDSSKEFVEYKITEWPTATGDVVLANDDLYVKRYERGCITFGKPVIWLSHEKASLNSLTYFNRPLLVD",
"R1AB_CVM2/1898-1999":"IKAQFRTFEKVEGVYTNFKLVGHSIAEKFNAKLGFDCNSPFTEYKITEWPTATGDVVLASDDLYVSRYSGGCVTFGKPVIWLGHEEASLKSLTYFNRPSVVC",
"R1AB_CVMA5/1951-2052":"IKAQFRTFEKVDGVYTNFKLVGHSIAEKLNAKLGFDCNSPFVEYKITEWPTATGDVVLASDDLYVSRYSSGCITFGKPVVWLGHEEASLKSLTYFNRPSVVC",
"R1AB_CVMJH/1950-2051":"IKAQFRTFEKVEGVYTNFKLVGHDIAEKLNAKLGFDCNSPFMEYKITEWPTATGDVVLASDDLYVSRYSGGCVTFGKPVIWRGHEEASLKSLTYFNRPSVVC",
"R1AB_MERS1/1837-1954":"PVTYSPATILAGSVYTNSCLVSSDGQPGGDAISLSFNNLLGFDSSKPVTKKYTYSFLPKEDGDVLLAEFDTYDPIYKNGAMYKGKPILWVNKASYDTNLNKFNRASLRQIFDVAPIEL",
"R1AB_SARS/1888-1998":"PIDLVPTQPLPNASFDNFKLTCSNTKFADDLNQMTGFTKPASRELSVTFFPDLNGDVVAIDYRHYSASFKKGAKLLHKPIVWHINQATTKTTFKPNTWCLRCLWSTKPVDT",
"R1AB_SARS2/1911-2021":"PIDLVPNQPYPNASFDNFKFVCDNIKFADDLNQLTGYKKPASRELKVTFFPDLNGDVVAIDYKHYTPSFKKGAKLLHKPIVWHVNNATNKATYKPNTWCIRCLWSTKPVET",
"R1A_CVBEN/1906-2007":"IKAQFKTFEKVDGVYTNFKLIGHTICDILNAKLGFDSSKEFVEYKVTEWPTATGDVVLATDDLYVKRYERGCITFGKPVIWLSHEQASLNSLTYFNRPLLVD",
"R1A_CVBLU/1906-2007":"IKAQFKTFEKVDGVYTNFKLIGHTICDILNAKLGFDSSKEFVEYKVTEWPTATGDVVLATDDLYVKRYERGCITFGKPVIWLSHEQASLNSLTYFNRPLLVD",
"R1A_CVBM/1906-2007":"IKAQFKTFEKVDGVYTNFKLIGHTVCDILNAKLGFDSSKEFVEYKVTEWPTATGDVVLATDDLYVKRYERGCITFGKPVIWLSHEQASLNSLTYFNRPLLVD",
"R1A_CVBQ/1906-2007":"IKAQFKTFEKVDGVYTNFKLIGHTVCDILNAKLGFDSSKEFVEYKVTEWPTATGDVVLATDDLYVKRYERGCITFGKPVIWLSHEQASLNSLTYFNRPLLVD",
"R1A_CVHN1/1992-2093":"IKAQFKPFAKVDGVYTNFKLVGHDICAQLNDKLGFNVDLPFVEYKVTVWPVATGDVVLASDDLYVKRYFKGCETFGKPVIWFCHDEASLNSLTYFNKPSFKS",
"R1A_CVHN2/1962-2063":"IKAQFKPFAKVDGVYTNFKLVGHDICAQLNDKLGFNVDLPFVEYKVTVWPVATGDVVLASDDLYVKRYFKGCETFGKPVIWLCHDEASLNSLTYFNKPSFKS",
"R1A_CVHN5/1942-2043":"IKAQFKPFAKVDGVYTNFKLVGHDICAQLNDKLGFNVDLPFVEYKVTVWPVATGDVVLASDDLYVKRYFKGCETFGKPVIWFCHDEASLNSLTYFNKPSFKS",
"R1A_CVHOC/1906-2007":"IKAQFKTFEKVDGVYTNFKLIGHTVCDSLNSKLGFDSSKEFVEYKITEWPTATGDVVLANDDLYVKRYERGCITFGKPVIWLSHEKASLNSLTYFNRPLLVD",
"R1A_CVM2/1898-1999":"IKAQFRTFEKVEGVYTNFKLVGHSIAEKFNAKLGFDCNSPFTEYKITEWPTATGDVVLASDDLYVSRYSGGCVTFGKPVIWLGHEEASLKSLTYFNRPSVVC",
"R1A_CVMA5/1951-2052":"IKAQFRTFEKVDGVYTNFKLVGHSIAEKLNAKLGFDCNSPFVEYKITEWPTATGDVVLASDDLYVSRYSSGCITFGKPVVWLGHEEASLKSLTYFNRPSVVC",
"R1A_CVMJH/1950-2051":"IKAQFRTFEKVEGVYTNFKLVGHDIAEKLNAKLGFDCNSPFMEYKITEWPTATGDVVLASDDLYVSRYSGGCVTFGKPVIWRGHEEASLKSLTYFNRPSVVC",
"R1A_MERS1/1837-1954":"PVTYSPATILAGSVYTNSCLVSSDGQPGGDAISLSFNNLLGFDSSKPVTKKYTYSFLPKEDGDVLLAEFDTYDPIYKNGAMYKGKPILWVNKASYDTNLNKFNRASLRQIFDVAPIEL",
"R1A_SARS/1888-1998":"PIDLVPTQPLPNASFDNFKLTCSNTKFADDLNQMTGFTKPASRELSVTFFPDLNGDVVAIDYRHYSASFKKGAKLLHKPIVWHINQATTKTTFKPNTWCLRCLWSTKPVDT",
"R1A_SARS2/1911-2021":"PIDLVPNQPYPNASFDNFKFVCDNIKFADDLNQLTGYKKPASRELKVTFFPDLNGDVVAIDYKHYTPSFKKGAKLLHKPIVWHVNNATNKATYKPNTWCIRCLWSTKPVET"}

Nsp3e_Seqs = [msa.MySeq(val, "PROTEIN") for val in Nsp3e_seqs.values()]

sm = msa.SubstMatrix()
# sm.create_submat(1, -1, "ACGT")
sm.read_submat_file("/content/bioinformatics_stockholm/msa/blosum62.mat", "\t")
aseq = msa.PairwiseAlignment(sm, -1)
ma_Nsp3e = msa.MultipleAlignment(Example_seqs, aseq)
the_sequences = ma_Nsp3e.align_consensus()
print(the_sequences)

## optional challenge problem
**Generate an amino acid frequency table like those discussed in the lecture. Use the multiple alignment data from the zinc finger example earlier in this section. Your matrix should have a row for every amino acid type and a column corresponding to each column in the multiple alignment. And elements of the matrix should have values corresponding to the amino acid frequency in that column.